In [2]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from fake_useragent import UserAgent
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
import pdfquery
from lxml import etree
import xml.etree.cElementTree as ET
import re
import sklearn.cluster
import distance
import string
import spacy
import nltk

In [4]:
def combine(files):
    first = None
    for filename in files:
        data = ET.parse(filename).getroot()
        if first is None:
            first = data
        else:
            first.extend(data)
    if first is not None:
        f = open('D:\FHSnotes\Thesis\Hansard\Parl45.xml', 'wb')
        f.write(ET.tostring(first))
        f.close()
        # return et.tostring(first)

combine(('D:\FHSnotes\Thesis\Hansard\Parl45.xml','D:\FHSnotes\Thesis\Hansard\Vol 559.xml','D:\FHSnotes\Thesis\Hansard\Vol 560.xml','D:\FHSnotes\Thesis\Hansard\Vol 561.xml','D:\FHSnotes\Thesis\Hansard\Vol 543.xml','D:\FHSnotes\Thesis\Hansard\Vol 562.xml','D:\FHSnotes\Thesis\Hansard\Vol 563.xml','D:\FHSnotes\Thesis\Hansard\Vol 564.xml','D:\FHSnotes\Thesis\Hansard\Vol 565.xml','D:\FHSnotes\Thesis\Hansard\Vol 566.xml','D:\FHSnotes\Thesis\Hansard\Vol 567.xml','D:\FHSnotes\Thesis\Hansard\Vol 568.xml','D:\FHSnotes\Thesis\Hansard\Vol 569.xml','D:\FHSnotes\Thesis\Hansard\Vol 570.xml','D:\FHSnotes\Thesis\Hansard\Vol 571.xml','D:\FHSnotes\Thesis\Hansard\Vol 572.xml','D:\FHSnotes\Thesis\Hansard\Vol 573.xml','D:\FHSnotes\Thesis\Hansard\Vol 574.xml','D:\FHSnotes\Thesis\Hansard\Vol 575.xml','D:\FHSnotes\Thesis\Hansard\Vol 576.xml','D:\FHSnotes\Thesis\Hansard\Vol 577.xml','D:\FHSnotes\Thesis\Hansard\Vol 578.xml','D:\FHSnotes\Thesis\Hansard\Vol 579.xml','D:\FHSnotes\Thesis\Hansard\Vol 580.xml'))

In [3]:
pdf_file = "D:\FHSnotes\Thesis\Hansard\Parl45.xml" # Change this to where you store the documents
exp = re.compile(r'<.*?>')
with open(pdf_file[0:-4]+".xml") as f:
    lines = ''.join(line for line in f.readlines())
    text_only = exp.sub('',lines).strip()

In [3]:
speakers_indices = [(m.start(0), m.end(0)) for m in re.finditer(r'[\r\n]+.{0,100}:', text_only)]

In [4]:
speakers = [text_only[a:b] for (a,b) in speakers_indices]
speakers

['\n     The Chief Commissioner then said:',
 '\n    members, which being done they took their seats in the House:',
 '\n     Hon. DOUG KIDD (NZ National---Kaikoura):',
 '\n     Rt Hon. JONATHAN HUNT (NZ Labour):',
 "\n    Donald Hobbs O'Connor Yates Duynhoven Hodgson Peck Dyson Hunt Pettis Teller:",
 '\n     Mr SPEAKER-ELECT:',
 '\n     Rt Hon. J B BOLGER (Prime Minister):',
 '\n     Rt Hon. HELEN CLARK (Leader of the Opposition):',
 '\n     JIM ANDERTON (Leader---The Alliance):',
 '\n     Hon. WINSTON PETERS (Leader---NZ First):',
 '\n     George Hawkins:',
 '\n     Hon. WINSTON PETERS:',
 '\n     Geoff Braybrooke:',
 '\n     Hon. Phil Goff:',
 '\n     Hon. WINSTON PETERS:',
 '\n     Hon. RICHARD PREBBLE (Leader---Act NZ):',
 '\n     Rt Hon. Helen Clark:',
 '\n     Hon. RICHARD PREBBLE:',
 '\n     Rt Hon. Helen Clark:',
 '\n     Hon. RICHARD PREBBLE:',
 '\n     Hon. PETER DUNNE (Leader---United NZ):',
 '\n     Rt Hon. JONATHAN HUNT (NZ Labour):',
 '\n     Mr SPEAKER-ELECT:',
 '\n    

In [5]:
len(speakers_indices)

101205

In [6]:
speeches = list()
indices_start = list()
indices_end = list()
for speaker_number in range(0,len(speakers),1):
    if speaker_number == len(speakers) - 1:
        index_end = len(text_only) - 1
    else:
        index_start = speakers_indices[speaker_number][1]
        index_end = speakers_indices[speaker_number+1][0]
    indices_start = indices_start + [index_start]
    indices_end = indices_end + [index_end]
    speeches = speeches +  [text_only[index_start:index_end]]
print(len(speakers),len(speeches),len(indices_start),len(indices_end))

101205 101205 101205 101205


In [7]:
df_speeches = pd.DataFrame.from_dict({"Speaker": speakers, "start": indices_start, "end": indices_end, "Speech": speeches})

In [8]:
df_speeches.head(50)

,Speaker,start,end,Speech
0,\n The Chief Commissioner then said:,1638,3764,"MICHAEL HARDIE BOYS, Governor-General. A PROC..."
1,"\n members, which being done they took thei...",3830,6041,"Arthur Anae, James Patrick Anderton, Donna Ly..."
2,\n Hon. DOUG KIDD (NZ National---Kaikoura):,6087,6150,"The Clerk, I declare my nomination for the of..."
3,\n Rt Hon. JONATHAN HUNT (NZ Labour):,6190,7209,"The Clerk, I declare my nomination for electi..."
4,\n Donald Hobbs O'Connor Yates Duynhoven Ho...,7290,7403,Field Keall Robertson Burton Majority for Hon...
5,\n Mr SPEAKER-ELECT:,7426,8678,"Honourable members, thank you for this appoin..."
6,\n Rt Hon. J B BOLGER (Prime Minister):,8720,10386,"Mr Speaker-elect, can I be the first to rise ..."
7,\n Rt Hon. HELEN CLARK (Leader of the Oppo...,10439,11466,"Mr Speaker-elect, I personally offer you cong..."
8,\n JIM ANDERTON (Leader---The Alliance):,11509,13742,"Mr Speaker-elect, might I say that that is on..."
9,\n Hon. WINSTON PETERS (Leader---NZ First):,13788,14802,"Mr Speaker-elect, I want to offer the congrat..."


In [9]:
df_speeches['important_speaker'] = [
    any([(speaker_word.isupper()) and (len(speaker_word) > 1) for speaker_word in speaker.split(" ")])
    for speaker in df_speeches['Speaker']
]

In [10]:
df_important_speeches = df_speeches[df_speeches.important_speaker]

In [79]:
exclude = set(string.punctuation)
df_speeches['speaker_name'] = [
    ' '.join([speaker_word for speaker_word in ''.join(ch for ch in speaker if ch not in exclude).split(" ")if((speaker_word.isupper()) and (len(speaker_word) > 1))])
    for speaker in df_speeches['Speaker']
]
df_speeches['speaker_name']
df_speeches.speaker_name.unique()

array(['', 'DOUG KIDD NZ', 'JONATHAN HUNT NZ', 'SPEAKERELECT', 'BOLGER',
       'HELEN CLARK', 'JIM ANDERTON', 'WINSTON PETERS',
       'RICHARD PREBBLE NZ', 'RICHARD PREBBLE', 'PETER DUNNE NZ',
       'SPEAKER', 'DON', 'MICHAEL CULLEN', 'GEOFF BRAYBROOKE NZ',
       'JUDITH TIZARD NZ', 'RICK BARKER NZ', 'RICK BARKER',
       'JANET MACKEY NZ', 'ROGER SOWRY', 'TREVOR MALLARD NZ',
       'DIANNE YATES NZ', 'MIKE MOORE NZ', 'MIKE MOORE', 'KEN SHIRLEY NZ',
       'KEN SHIRLEY', 'JIM GERARD NZ', 'WYATT CREECH', 'DEREK QUIGLEY NZ',
       'BILL BIRCH', 'MAURICE WILLIAMSON', 'PAUL EAST', 'MARK PECK NZ',
       'MARK PECK', 'TEMPORARY SPEAKER', 'TAU HENARE', 'SANDRA LEE',
       'JACK ELDER NZ', 'JACK ELDER', 'STEVE MAHAREY NZ', 'STEVE MAHAREY',
       'PETER NZ', 'PETER', 'PHIL GOFF NZ', 'PHIL GOFF', 'TONY RYALL NZ',
       'ANNETTE KING NZ', 'TREVOR MALLARD', 'MARIE HASLER NZ',
       'BRIAN NEESON NZ', 'BILL ENGLISH', 'TONY STEEL NZ',
       'LIANNE DALZIEL NZ', 'DOUG WOOLERTON NZ', 'ROGER

In [102]:
test = pd.DataFrame.from_dict({"speaker_name": df_speeches['speaker_name']})
test = test.astype(str)
test1 = test.apply(lambda x: x.str.split().str[0])
test2 = test.apply(lambda x: x.str.split().str[1])
test2 = test2.fillna('')
test = test1 + ' ' + test2
test = test.astype(str)
df_speeches['speaker_name'] = test['speaker_name']
df_speeches.speaker_name.unique()

array(['nan ', 'DOUG KIDD', 'JONATHAN HUNT', 'SPEAKERELECT ', 'BOLGER ',
       'HELEN CLARK', 'JIM ANDERTON', 'WINSTON PETERS', 'RICHARD PREBBLE',
       'PETER DUNNE', 'SPEAKER ', 'DON ', 'MICHAEL CULLEN',
       'GEOFF BRAYBROOKE', 'JUDITH TIZARD', 'RICK BARKER', 'JANET MACKEY',
       'ROGER SOWRY', 'TREVOR MALLARD', 'DIANNE YATES', 'MIKE MOORE',
       'KEN SHIRLEY', 'JIM GERARD', 'WYATT CREECH', 'DEREK QUIGLEY',
       'BILL BIRCH', 'MAURICE WILLIAMSON', 'PAUL EAST', 'MARK PECK',
       'TEMPORARY SPEAKER', 'TAU HENARE', 'SANDRA LEE', 'JACK ELDER',
       'STEVE MAHAREY', 'PETER NZ', 'PETER ', 'PHIL GOFF', 'TONY RYALL',
       'ANNETTE KING', 'MARIE HASLER', 'BRIAN NEESON', 'BILL ENGLISH',
       'TONY STEEL', 'LIANNE DALZIEL', 'DOUG WOOLERTON', 'ROGER MAXWELL',
       'SIMON UPTON', 'GERRY BROWNLEE', 'JILL WHITE', 'GEORGE HAWKINS',
       'DEPUTY SPEAKER', 'BOB SIMCOCK', 'JOHN CARTER', 'JENNY BLOXHAM',
       'JEANETTE FITZSIMONS', 'CLEM SIMICH', 'PETER BROWN',
       'JENNY SHI

In [103]:
lab96 = "Sutton Tizard Barnett Sutherland Hodgson Cullen Mallard Peck Mackey Kelly Field Robertson Hawkins Braybrooke Goff Duynhoven Keall Clark Maharey Swain King Burton Moore O'Connor OConnor Pettis Samuels Dalziel Gosche Hunt Mahuta White Hobbs Hawke Yates Dyson Turia"
lab96E = "Sutton Tizard Barnett Sutherland Hodgson Cullen Mallard Peck Mackey Kelly Field Robertson Hawkins Braybrooke Goff Duynhoven Keall Clark Maharey Swain King Burton Moore O'Connor OConnor Pettis"
lab96L = "Samuels Dalziel Gosche Hunt Mahuta White Hobbs Hawke Yates Dyson Turia"
nat96 = "McCully Carter Ryall English McLean Fletcher Steel Simcock Kyd Brownlee Kidd Luxton Vernon Smith Mapp Revell Carter Herlihy Williamson Birch Shipley Marshall Smith Bradford Simich Bolger Neeson Creech Hasler Banks Ardern Don McKinnon East Graham Heuheu Regan Upton McLauchlan Sowry Gerard Anae Roy Gresham Maxwell Wong"
nat96E = "McCully Carter Ryall English McLean Fletcher Steel Simcock Kyd Brownlee Kidd Luxton Vernon Smith Mapp Revell Carter Herlihy Williamson Birch Shipley Marshall Smith Bradford Simich Bolger Neeson Creech Hasler Banks Ardern"
nat96L = "McKinnon East Graham Heuheu Regan Upton McLauchlan Sowry Gerard Anae Roy Gresham Maxwell Wong"
stringtotest = "HELEN CLARK"
settotest = set(stringtotest.split())

In [104]:
test['name'] = [
    'SUTTON (LAB)' if 'SUTTON' in speaker_name
    else 'TIZARD (LAB)' if 'TIZARD' in speaker_name
    else 'BARNETT (LAB)' if 'BARNETT' in speaker_name
    else 'SUTHERLAND (LAB)' if 'SUTHERLAND' in speaker_name
    else 'HODGSON (LAB)' if 'HODGSON' in speaker_name
    else 'CULLEN (LAB)' if 'CULLEN' in speaker_name
    else 'MALLARD (LAB)' if 'MALLARD' in speaker_name
    else 'PECK (LAB)' if 'PECK' in speaker_name
    else 'MACKEY (LAB)' if 'MACKEY' in speaker_name
    else 'KELLY (LAB)' if 'KELLY' in speaker_name
    else 'FIELD (LAB)' if 'FIELD' in speaker_name
    else 'ROBERTSON (LAB)' if 'ROBERTSON' in speaker_name
    else 'HAWKINS (LAB)' if 'HAWKINS' in speaker_name
    else 'BRAYBROOKE (LAB)' if 'BRAYBROOKE' in speaker_name
    else 'GOFF (LAB)' if 'GOFF' in speaker_name
    else 'DUYNHOVEN (LAB)' if 'DUYNHOVEN' in speaker_name
    else 'KEALL (LAB)' if 'KEALL' in speaker_name
    else 'CLARK (LAB)' if 'CLARK' in speaker_name
    else 'MAHAREY (LAB)' if 'MAHAREY' in speaker_name
    else 'SWAIN (LAB)' if 'SWAIN' in speaker_name
    else 'KING (LAB)' if 'KING' in speaker_name
    else 'BURTON (LAB)' if 'BURTON' in speaker_name
    else 'MOORE (LAB)' if 'MOORE' in speaker_name
    else 'OCONNOR (LAB)' if 'OCONNOR' in speaker_name
    else 'PETTIS (LAB)' if 'PETTIS' in speaker_name
    else 'SAMUELS (LAB)' if 'SAMUELS' in speaker_name
    else 'DALZIEL (LAB)' if 'DALZIEL' in speaker_name
    else 'GOSCHE (LAB)' if 'GOSCHE' in speaker_name
    else 'HUNT (LAB)' if 'HUNT' in speaker_name
    else 'MAHUTA (LAB)' if 'MAHUTA' in speaker_name
    else 'WHITE (LAB)' if 'WHITE' in speaker_name
    else 'HOBBS (LAB)' if 'HOBBS' in speaker_name
    else 'HAWKE (LAB)' if 'HAWKE' in speaker_name
    else 'YATES (LAB)' if 'YATES' in speaker_name
    else 'DYSON (LAB)' if 'DYSON' in speaker_name
    else 'TURI (LAB)' if 'TURI' in speaker_name
    else 'MCCULLY (NAT)' if 'MURRAY' in speaker_name
    else 'JOHN CARTER (NAT)' if 'JOHN CARTER' in speaker_name
    else 'DAVID CARTER (NAT)' if 'DAVID CARTER' in speaker_name
    else 'RYALL (NAT)' if 'RYALL' in speaker_name
    else 'ENGLISH (NAT)' if 'ENGLISH' in speaker_name
    else 'MCLEAN (NAT)' if 'MCLEAN' in speaker_name
    else 'FLETCHER (NAT)' if 'FLETCHER' in speaker_name
    else 'STEEL (NAT)' if 'STEEL' in speaker_name
    else 'SIMCOCK (NAT)' if 'SIMCOCK' in speaker_name
    else 'KYD (NAT)' if 'KYD' in speaker_name
    else 'BROWNLEE (NAT)' if 'BROWNLEE' in speaker_name
    else 'KIDD (NAT)' if 'KIDD' in speaker_name
    else 'LUXTON (NAT)' if 'LUXTON' in speaker_name
    else 'VERNON (NAT)' if 'VERNON' in speaker_name
    else 'NICK SMITH (NAT)' if 'NICK SMITH' in speaker_name
    else 'LOCKWOOD SMITH (NAT)' if 'SMITH' in speaker_name
    else 'MAPP (NAT)' if 'MAPP' in speaker_name
    else 'REVELL (NAT)' if 'REVELL' in speaker_name
    else 'HERLIHY (NAT)' if 'HERLIHY' in speaker_name
    else 'WILLIAMSON (NAT)' if 'WILLIAMSON' in speaker_name
    else 'BIRCH (NAT)' if 'MCCULLY' in speaker_name
    else 'SHIPLEY (NAT)' if 'SHIPLEY' in speaker_name
    else 'MARSHALL (NAT)' if 'MARSHALL' in speaker_name
    else 'BRADFORD (NAT)' if 'BRADFORD' in speaker_name
    else 'SIMICH (NAT)' if 'SIMICH' in speaker_name
    else 'BOLGER (NAT)' if 'BOLGER' in speaker_name
    else 'NEESON (NAT)' if 'NEESON' in speaker_name
    else 'CREECH (NAT)' if 'CREECH' in speaker_name
    else 'HASLER (NAT)' if 'HASLER' in speaker_name
    else 'BANKS (NAT)' if 'BANKS' in speaker_name
    else 'ARDERN (NAT)' if 'ARDERN' in speaker_name
    else 'MCKINNON (NAT)' if 'DON' in speaker_name
    else 'EAST (NAT)' if 'EAST' in speaker_name
    else 'GRAHAM (NAT)' if 'GRAHAM' in speaker_name
    else 'HEUHEU (NAT)' if 'HEUHEU' in speaker_name
    else 'REGAN (NAT)' if 'REGAN' in speaker_name
    else 'UPTON (NAT)' if 'UPTON' in speaker_name
    else 'MCLAUCHLAN (NAT)' if 'MCLAUCHLAN' in speaker_name
    else 'SOWRY (NAT)' if 'SOWRY' in speaker_name
    else 'GERARD (NAT)' if 'GERARD' in speaker_name
    else 'ANAE (NAT)' if 'ANAE' in speaker_name
    else 'ROY (NAT)' if 'ROY' in speaker_name
    else 'GRESHAM (NAT)' if 'GRESHAM' in speaker_name
    else 'MAXWELL (NAT)' if 'MAXWELL' in speaker_name
    else 'WONG (NAT)' if 'WONG' in speaker_name
    else ''
    for speaker_name in test['speaker_name']
]
df_speeches['name'] = test['name']
test.name.unique()

array(['', 'KIDD (NAT)', 'HUNT (LAB)', 'BOLGER (NAT)', 'CLARK (LAB)',
       'MCKINNON (NAT)', 'CULLEN (LAB)', 'BRAYBROOKE (LAB)',
       'TIZARD (LAB)', 'MACKEY (LAB)', 'SOWRY (NAT)', 'MALLARD (LAB)',
       'YATES (LAB)', 'MOORE (LAB)', 'GERARD (NAT)', 'CREECH (NAT)',
       'WILLIAMSON (NAT)', 'EAST (NAT)', 'PECK (LAB)', 'MAHAREY (LAB)',
       'GOFF (LAB)', 'RYALL (NAT)', 'KING (LAB)', 'HASLER (NAT)',
       'NEESON (NAT)', 'ENGLISH (NAT)', 'STEEL (NAT)', 'DALZIEL (LAB)',
       'MAXWELL (NAT)', 'UPTON (NAT)', 'BROWNLEE (NAT)', 'WHITE (LAB)',
       'HAWKINS (LAB)', 'SIMCOCK (NAT)', 'JOHN CARTER (NAT)',
       'SIMICH (NAT)', 'SHIPLEY (NAT)', 'DAVID CARTER (NAT)',
       'LUXTON (NAT)', 'ANAE (NAT)', 'MAHUTA (LAB)', 'HAWKE (LAB)',
       'KYD (NAT)', 'BRADFORD (NAT)', 'REGAN (NAT)', 'HOBBS (LAB)',
       'MCCULLY (NAT)', 'SUTTON (LAB)', 'MAPP (NAT)', 'SWAIN (LAB)',
       'ROY (NAT)', 'GRAHAM (NAT)', 'HODGSON (LAB)', 'NICK SMITH (NAT)',
       'SAMUELS (LAB)', 'HEUHEU (NAT)', 'LOCK

In [105]:
labset96 = set(
    [
        word.upper()
        for word in lab96.split()
    ]
)
natset96 = set(
    [
        word.upper()
        for word in nat96.split()
    ]
)

if labset96.intersection(settotest) is not None:
    party='Labour'
    
party

'Labour'

In [106]:
stringtotest = "HELEN CLARK"
settotest = set(stringtotest.split())
len(labset96.intersection(settotest)) > 0

True

In [107]:
df_speeches['party'] = [
    'Labour' if '(LAB)' in name
    else 'National' if '(NAT)' in name
    else ""
    for name in df_speeches['name']
]

In [109]:
df_speeches[['speaker_name','name','party']]

,speaker_name,name,party
0,nan,,
1,nan,,
2,DOUG KIDD,KIDD (NAT),National
3,JONATHAN HUNT,HUNT (LAB),Labour
4,nan,,
...,...,...,...
101200,nan,,
101201,MICHAEL CULLEN,CULLEN (LAB),Labour
101202,nan,,
101203,MICHAEL CULLEN,CULLEN (LAB),Labour


In [110]:
pd.options.display.max_colwidth = 500
pd.set_option('display.max_rows', 500)
df_important_speeches = df_speeches[df_speeches.important_speaker]
df_important_speeches.head(50)

,Speaker,start,end,Speech,important_speaker,speaker_name,party,name
2,\n Hon. DOUG KIDD (NZ National---Kaikoura):,6087,6150,"The Clerk, I declare my nomination for the office of Speaker.",True,DOUG KIDD,National,KIDD (NAT)
3,\n Rt Hon. JONATHAN HUNT (NZ Labour):,6190,7209,"The Clerk, I declare my nomination for election as Speaker. \n A personal vote was called for on the question of the election of the Speaker. \n Hon. Doug Kidd 70 Anae English McLauchlan Shipley Awatere Huata Fletcher McLean Shirley Banks Gerard Mapp Simcock Batten Graham Mark Simich Birch Gresham Marshall Smith, L. Bloxham Hasler Maxwell Smith, N. Bolger Henare Morgan Steel Bradford Herlihy Morris te Heuheu Brown Hide Neeson Upton Brownlee Jennings Newman Vernon Carter D Kidd O'Reg...",True,JONATHAN HUNT,Labour,HUNT (LAB)
5,\n Mr SPEAKER-ELECT:,7426,8678,"Honourable members, thank you for this appointment---especially those who voted for me. As to those who did not, I want to assure them most sincerely that I will preside without fear or favour, and my impartiality will be assured. With a modicum of cooperation, I believe that this first MMP Parliament will be seen as a watershed. We have now moved into a new system. We have trialled the Standing Orders to operate. Generally they were a very considerable advance and, I believe, a success, bu...",True,SPEAKERELECT,,
6,\n Rt Hon. J B BOLGER (Prime Minister):,8720,10386,"Mr Speaker-elect, can I be the first to rise to congratulate you on assuming the considerable responsibilities of the Speakership of this House. Now that the vote has been concluded, I am sure I speak on behalf of all members when I say that we will give you our support on most occasions here and will be guided by your advice, although on occasions we might not hear it. Mr Speaker-elect, as you yourself have said, we enter a new era of MMP. We enter it in a House that is much larger, and th...",True,BOLGER,National,BOLGER (NAT)
7,\n Rt Hon. HELEN CLARK (Leader of the Opposition):,10439,11466,"Mr Speaker-elect, I personally offer you congratulations on your election to the position. I express my deep regret that the father of the House was not elected, and I express my considerable concern that the former Deputy Speaker, who served with distinction in the previous Parliament, was not able to have his name put forward. \n I note that the report of the Standing Orders Committee on the new Standing Orders for this Parliament anticipated that in the spirit of MMP there should be ...",True,HELEN CLARK,Labour,CLARK (LAB)
8,\n JIM ANDERTON (Leader---The Alliance):,11509,13742,"Mr Speaker-elect, might I say that that is one of the prices that you have to pay for coalition with the Government, too. No one knows more than do minority parties in this House the importance of the objectivity and impartiality of the Speaker. That is a long, and I think proud, tradition in this House. We fully expect you to uphold it and we know that you will. Therefore we congratulate you on your election. \n The Alliance supported the nomination of the Rt Hon. Jonathan Hunt. We did...",True,JIM ANDERTON,,
9,\n Hon. WINSTON PETERS (Leader---NZ First):,13788,14802,"Mr Speaker-elect, I want to offer the congratulations of New Zealand First on your appointment and to make it very clear to the Labour Party and to the Alliance that in discussions with the Labour Party we said from day one that those two positions would be its to prefer and it would be the same case with National, and that is the position that we hold right now. So to begin this new era with such acrimony will not, in my view, be good for this country. \n I remind the Labour Party that...",True,WINSTON PETERS,,
11,\n Hon. WINSTON PETERS:,14865,15266,"If the member insists upon it, but that is not our intention. Mr Speaker-elect, I think you are eminently qualified for this job---for a start, you are a lawyer; a very successful one, with long parliamentary experience. I believe that even though you may be described

In [111]:
df_important_speeches.to_csv("importantspeeches45.csv")

In [20]:
## https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
en_stop = set(nltk.corpus.stopwords.words('english'))
en_stop.update(['people','minister','ministers','would','house','member','government','committee','committees','office','offices','matter','matters','clause','clauses','agree','agreement','disagree','disagreement','opposition','member','point','order','answer','answers','question','speech','speaker','hear','department','members','objection','object','amend','amendment','amendments','vote','voting','votes','aye','no','think','thought','thinking','voted','ought','leader','prime','debate','debates','issue','issues','whether','support','supports','supporting','oppose','opposes','opposing','supported','opposed','zealand','process','legislation','legislator','legislators','party','parties','caucus','bring','bringing','brings','brought','labour','national','alliance','change','might','changes','changing','move','policy','submission','group','receive','report','submissions','groups','receives','reports','parliament'])
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [21]:
text_data = []

for speech in df_important_speeches['Speech']:
        tokens = prepare_text_for_lda(speech)
        text_data.append(tokens)

In [22]:
len(text_data)

66136

In [23]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [24]:
import gensim
NUM_TOPICS = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=10)

In [25]:
import os
from gensim.models.wrappers import LdaMallet

os.environ['MALLET_HOME'] = 'C:\\Users\\matth\\Downloads\\mallet-2.0.8\\mallet-2.0.8'
mallet_path = 'C:\\Users\\matth\\Downloads\\mallet-2.0.8\\mallet-2.0.8\\bin\\mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=dictionary)
mtopics = ldamallet.print_topics(num_words=10)
for mtopic in mtopics:
    print(mtopic)

(0, '0.022*"person" + 0.018*"police" + 0.018*"court" + 0.011*"justice" + 0.010*"security" + 0.008*"evidence" + 0.008*"crime" + 0.008*"power" + 0.007*"case" + 0.007*"officer"')
(1, '0.032*"country" + 0.014*"election" + 0.012*"coalition" + 0.011*"today" + 0.010*"political" + 0.010*"zealander" + 0.008*"stand" + 0.008*"thing" + 0.008*"force" + 0.007*"peter"')
(2, '0.025*"business" + 0.025*"industry" + 0.022*"company" + 0.015*"board" + 0.014*"country" + 0.013*"market" + 0.010*"state" + 0.010*"price" + 0.009*"trade" + 0.009*"farmer"')
(3, '0.026*"child" + 0.023*"school" + 0.022*"family" + 0.021*"education" + 0.018*"community" + 0.018*"social" + 0.016*"young" + 0.015*"benefit" + 0.013*"student" + 0.013*"programme"')
(4, '0.039*"maori" + 0.011*"crown" + 0.010*"claim" + 0.010*"affairs" + 0.010*"colleague" + 0.009*"property" + 0.009*"terms" + 0.009*"island" + 0.008*"settlement" + 0.008*"treaty"')
(5, '0.051*"health" + 0.029*"service" + 0.029*"services" + 0.025*"public" + 0.015*"commission" + 0.0

In [1]:
for mtopic in mtopics:
    print(mtopic)

NameError: name 'mtopics' is not defined

In [26]:
df_important_speeches = df_important_speeches.reset_index(drop=True)
df_important_speeches.head(500)

,Speaker,start,end,Speech,important_speaker,speaker_name,party
0,\n Hon. DOUG KIDD (NZ National---Kaikoura):,6087,6150,"The Clerk, I declare my nomination for the office of Speaker.",True,DOUG KIDD NZ,National
1,\n Rt Hon. JONATHAN HUNT (NZ Labour):,6190,7209,"The Clerk, I declare my nomination for election as Speaker. \n A personal vote was called for on the question of the election of the Speaker. \n Hon. Doug Kidd 70 Anae English McLauchlan Shipley Awatere Huata Fletcher McLean Shirley Banks Gerard Mapp Simcock Batten Graham Mark Simich Birch Gresham Marshall Smith, L. Bloxham Hasler Maxwell Smith, N. Bolger Henare Morgan Steel Bradford Herlihy Morris te Heuheu Brown Hide Neeson Upton Brownlee Jennings Newman Vernon Carter D Kidd O'Reg...",True,JONATHAN HUNT NZ,
2,\n Mr SPEAKER-ELECT:,7426,8678,"Honourable members, thank you for this appointment---especially those who voted for me. As to those who did not, I want to assure them most sincerely that I will preside without fear or favour, and my impartiality will be assured. With a modicum of cooperation, I believe that this first MMP Parliament will be seen as a watershed. We have now moved into a new system. We have trialled the Standing Orders to operate. Generally they were a very considerable advance and, I believe, a success, bu...",True,SPEAKERELECT,
3,\n Rt Hon. J B BOLGER (Prime Minister):,8720,10386,"Mr Speaker-elect, can I be the first to rise to congratulate you on assuming the considerable responsibilities of the Speakership of this House. Now that the vote has been concluded, I am sure I speak on behalf of all members when I say that we will give you our support on most occasions here and will be guided by your advice, although on occasions we might not hear it. Mr Speaker-elect, as you yourself have said, we enter a new era of MMP. We enter it in a House that is much larger, and th...",True,BOLGER,National
4,\n Rt Hon. HELEN CLARK (Leader of the Opposition):,10439,11466,"Mr Speaker-elect, I personally offer you congratulations on your election to the position. I express my deep regret that the father of the House was not elected, and I express my considerable concern that the former Deputy Speaker, who served with distinction in the previous Parliament, was not able to have his name put forward. \n I note that the report of the Standing Orders Committee on the new Standing Orders for this Parliament anticipated that in the spirit of MMP there should be ...",True,HELEN CLARK,Labour
5,\n JIM ANDERTON (Leader---The Alliance):,11509,13742,"Mr Speaker-elect, might I say that that is one of the prices that you have to pay for coalition with the Government, too. No one knows more than do minority parties in this House the importance of the objectivity and impartiality of the Speaker. That is a long, and I think proud, tradition in this House. We fully expect you to uphold it and we know that you will. Therefore we congratulate you on your election. \n The Alliance supported the nomination of the Rt Hon. Jonathan Hunt. We did...",True,JIM ANDERTON,
6,\n Hon. WINSTON PETERS (Leader---NZ First):,13788,14802,"Mr Speaker-elect, I want to offer the congratulations of New Zealand First on your appointment and to make it very clear to the Labour Party and to the Alliance that in discussions with the Labour Party we said from day one that those two positions would be its to prefer and it would be the same case with National, and that is the position that we hold right now. So to begin this new era with such acrimony will not, in my view, be good for this country. \n I remind the Labour Party that...",True,WINSTON PETERS,
7,\n Hon. WINSTON PETERS:,14865,15266,"If the member insists upon it, but that is not our intention. Mr Speaker-elect, I think you are eminently qualified for this job---for a start, you are a lawyer; a very successful one, with long parliamentary experience. I believe that even though you may be described as an old Tory, you have shown for many years those Whi

In [27]:
def format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=df_important_speeches.dropna()['Speech']):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=df_important_speeches.dropna()['Speech'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


In [28]:
df_dominant_topic['category'] = [
    'Justice' if topic == 0
    else 'Electoral' if topic == 1
    else 'Fiscal' if topic == 2
    else 'Education' if topic == 3
    else 'International' if topic == 4
    else 'Healthcare' if topic == 5
    else 'Civic' if topic == 6
    else 'Monetary' if topic == 7
    else 'Nonspecific' if topic == 8
    else 'Nonspecific2' if topic == 9
    else ""
    for topic in df_dominant_topic['Dominant_Topic']
]
df_dominant_topic['party'] = df_important_speeches.dropna()['party']

In [32]:
df_dominant_topic.to_csv("dominanttopic45.csv")
df_dominant_topic.head(70000)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,category,party,textblob,sentiment
0,0,9.0,0.1426,"raise, leave, agree, paper, question, sitting, comment, motion, supplementary, standing","The Clerk, I declare my nomination for the office of Speaker.",Nonspecific2,National,"( , T, h, e, , C, l, e, r, k, ,, , I, , d, e, c, l, a, r, e, , m, y, , n, o, m, i, n, a, t, i, o, n, , f, o, r, , t, h, e, , o, f, f, i, c, e, , o, f, , S, p, e, a, k, e, r, ., )",0.000000
1,1,4.0,0.3636,"maori, crown, claim, affairs, colleague, property, terms, island, settlement, treaty","The Clerk, I declare my nomination for election as Speaker. \n A personal vote was called for on the question of the election of the Speaker. \n Hon. Doug Kidd 70 Anae English McLauchlan Shipley Awatere Huata Fletcher McLean Shirley Banks Gerard Mapp Simcock Batten Graham Mark Simich Birch Gresham Marshall Smith, L. Bloxham Hasler Maxwell Smith, N. Bolger Henare Morgan Steel Bradford Herlihy Morris te Heuheu Brown Hide Neeson Upton Brownlee Jennings Newman Vernon Carter D Kidd O'Reg...",International,,"( , T, h, e, , C, l, e, r, k, ,, , I, , d, e, c, l, a, r, e, , m, y, , n, o, m, i, n, a, t, i, o, n, , f, o, r, , e, l, e, c, t, i, o, n, , a, s, , S, p, e, a, k, e, r, ., , \n, , , , , , A, , p, e, r, s, o, n, a, l, , v, o, t, e, , w, a, s, , c, a, l, l, e, d, , f, o, r, , o, n, ...)",0.000000
2,2,4.0,0.1753,"maori, crown, claim, affairs, colleague, property, terms, island, settlement, treaty","Honourable members, thank you for this appointment---especially those who voted for me. As to those who did not, I want to assure them most sincerely that I will preside without fear or favour, and my impartiality will be assured. With a modicum of cooperation, I believe that this first MMP Parliament will be seen as a watershed. We have now moved into a new system. We have trialled the Standing Orders to operate. Generally they were a very considerable advance and, I believe, a success, bu...",International,,"( , H, o, n, o, u, r, a, b, l, e, , m, e, m, b, e, r, s, ,, , t, h, a, n, k, , y, o, u, , f, o, r, , t, h, i, s, , a, p, p, o, i, n, t, m, e, n, t, -, -, -, e, s, p, e, c, i, a, l, l, y, , t, h, o, s, e, , w, h, o, , v, o, t, e, d, , f, o, r, , m, e, ., , A, s, , t, o, , t, h, o, s, e, ...)",0.265423
3,3,4.0,0.1520,"maori, crown, claim, affairs, colleague, property, terms, island, settlement, treaty","Mr Speaker-elect, can I be the first to rise to congratulate you on assuming the considerable responsibilities of the Speakership of this House. Now that the vote has been concluded, I am sure I speak on behalf of all members when I say that we will give you our support on most occasions here and will be guided by your advice, although on occasions we might not hear it. Mr Speaker-elect, as you yourself have said, we enter a new era of MMP. We enter it in a House that is much larger, and th...",International,National,"( , M, r, , S, p, e, a, k, e, r, -, e, l, e, c, t, ,, , c, a, n, , I, , b, e, , t, h, e, , f, i, r, s, t, , t, o, , r, i, s, e, , t, o, , c, o, n, g, r, a, t, u, l, a, t, e, , y, o, u, , o, n, , a, s, s, u, m, i, n, g, , t, h, e, , c, o, n, s, i, d, e, r, a, b, l, e, , r, e, s, p, o, ...)",0.132522
4,4,9.0,0.1780,"raise, leave, agree, paper, question, sitting, comment, motion, supplementary, standing","Mr Speaker-elect, I personally offer you congratulations on your election to the position. I express my deep regret that the father of the House was not elected, and I express my considerable concern that the former Deputy Speaker, who served with distinction in the previous Parliament, was not able to have his name put forward. \n I note that the report of the Standing Orders Committee on the new Standing Orders for this Parliament anticipated that in the spirit of MMP there should be ...",Nonspecific2,Labour,"( , M, r, , S, p, e, a, k, e, r, -, e, l, e, c, t, ,, , I, , p, e, r, s, o, n, a, l, l, y, , o, f, f, e, r, , y, o, u, , c, o, n, g, r, a, t, u, l

In [30]:
from textblob import TextBlob
df_dominant_topic['textblob'] = [
    TextBlob(text)
    for text in df_dominant_topic['Text']
]

In [31]:
df_dominant_topic['sentiment'] = [
    textblob.sentiment.polarity
    for textblob in df_dominant_topic['textblob']
]

In [64]:
n_instances = 100
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
len(subj_docs), len(obj_docs)
subj_docs[0]

train_subj_docs = subj_docs[:80]
test_subj_docs = subj_docs[80:100]
train_obj_docs = obj_docs[:80]
test_obj_docs = obj_docs[80:100]
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
len(unigram_feats)
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.8
F-measure [obj]: 0.8
F-measure [subj]: 0.8
Precision [obj]: 0.8
Precision [subj]: 0.8
Recall [obj]: 0.8
Recall [subj]: 0.8


In [53]:
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist

In [45]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [47]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [51]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens
stop_words = stopwords.words('english')

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [54]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)
freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [54]:
match_indices = list()
index=0
for i in range(0,text_only.count("environmental")):
    print(i)
    index = text_only.find("environmental",index+10)
    match_indices.append(index)
    identified_string = text_only[index:index+1000]
    print(identified_string)

0
environmental advantages contributes to the physical wellbeing of our citizens, provides benefits to producers and marketers of products from farms, forests and fisheries, and contributes to our growing inbound tourism. 
     New Zealand will continue to promote and support action on global environmental issues. At home, legislation will be promoted to reform the law to manage hazardous substances and new organisms, and to contain risks associated with them. 
     New Zealanders have a right to safe and secure communities, where they feel and are secure from unlawful and irresponsible actions of other citizens. My Government is committed to implementing a national crime prevention strategy. This will emphasise the prevention of criminal and antisocial actions, while recognising the importance of active policies to relieve stress in families. 
     My Government will integrate energy and environmental policy as a priority. Energy sector reforms will be continued. The challenge is to e

In [55]:
text_only.count("environmental")

923

In [56]:
text_only.find("environmental")

51890

In [57]:
text_only[text_only.find("environment"):text_only.find("environment")+100]

'environment. There will be no room for individuals who seek to use this House as a vehicle for self-'

In [112]:
df_dominant_topic44=pd.read_csv(r"C:\Users\matth\Downloads\dominanttopic.csv")
df_important_speeches44=pd.read_csv(r"C:\Users\matth\Downloads\importantspeeches44.csv")

In [113]:
df_dominant_topic45=pd.read_csv(r"C:\Users\matth\Downloads\dominanttopic45.csv")
df_important_speeches45=pd.read_csv(r"C:\Users\matth\Downloads\importantspeeches45.csv")

In [114]:
df_dominant_topic45=df_dominant_topic45.drop(columns='party')
df_dominant_topic44=df_dominant_topic44.drop(columns='party')

In [115]:
len(df_important_speeches44)

46751

In [116]:
df_speeches_master44 = pd.concat([df_dominant_topic44, df_important_speeches44], axis=1, sort=False)

In [117]:
df_speeches_master44=df_speeches_master44.drop(columns=['Text','Unnamed: 0','Document_No','Dominant_Topic','Topic_Perc_Contrib','Keywords','textblob','Speaker','end','important_speaker'])
df_speeches_master44

,category,sentiment,start,Speech,speaker_name,party,name
0,International,0.111077,5166,"I move, That Peter Wilfred Tapsell, the member for Eastern Maori, do take the chair of the House as Speaker. I am honoured and privileged to move this motion, and in doing so I am conscious of the historic significance of this nomination. We who have been elected to serve our country in this Parliament do so at a crucial time in our history. New Zealand has decided to change the method by which MPs are elected, and we in this House will be expected to reflect the new mood in the community i...",JOHN CARTER,National,JOHN CARTER (NAT)
1,Nonspecific,0.000000,6866,"E tu tautoko ana te motini kua whakatakotohia ai i tenei o nga mema o te Tai Tokerau, ara, kia tu ai te mema o te Tai Rawhiti ki runga ki tenei turanga teitei. He tika tonu he tuatahi te honore kua homai nei kia ngai taua ki te iwi Maori, ara, mo Aotearoa katoa. Tuarua, kei te tautoko i nga hau e wha o te motu, tae atu ano ki te Whare nei. \n He mihi kau ake ki te iwi o Te Arawa---Te Arawa waka mai i Maketu ki Tongariro, otira, tena koutou mo te taonga nui kua utaina nei ki runga ki a k...",WETERE,Labour,WETERE (LAB)
2,Nonspecific2,0.000000,10551,Does the member propose to submit to the judgment of the House?,CLERK THE,NaN,NaN
3,Nonspecific2,0.000000,10657,I should like to express my sense of the honour proposed to be conferred on me and to submit myself to the judgment of the House.,PETER TAPSELL,Labour,TAPSELL (LAB)
4,Nonspecific2,0.000000,10813,Are there any further proposals?,CLERK THE,NaN,NaN
...,...,...,...,...,...,...,...
46746,Nonspecific,0.026726,62292998,"No, this is not a valedictory---I know that Doug Graham was trying out my seat half an hour ago! It is time for a little bit of last housekeeping. I seek the leave of the House for Government, private, and local Bills on the Order Paper awaiting their second reading, with the exception of the New Zealand Courts Structure Bill and the Producer Board Acts Reform Bill, to be read a second time pro forma, and be referred to the appropriate select committee for consideration.",DON,National,MCKINNON (NAT)
46747,Nonspecific2,0.000000,62294334,"Leave is sought for that purpose. Is there any objection? There appears to be none. \n Bills read a second time, and referred to select committees. \n SELECT COMMITTEES \n Extension of Reporting Time",SPEAKER,NaN,NaN
46748,Healthcare,0.023810,62294593,"I seek leave for the following Bills before select committees to have the date by which they must be finally reported to the House extended to 30 June 1997: Degrees of Murder Bill, Health and Safety in Employment Amendment Bill, Irradiation Plant Public Consultation Bill, Limits on Campaign Spending for Candidates in Local Elections Bill, Medical Assurance Society Members' Trust (Exemption from Perpetuities) Bill, Osteopaths Bill, Repeal of Asset Testing for Older People Requiring Disabilit...",DON,National,MCKINNON (NAT)
46749,Nonspecific,0.100000,62295312,Leave is sought for that matter. Is there any objection to that course of action? There appears to be none. \n VALEDICTORIES \n Debate resumed.,SPEAKER,NaN,NaN


In [118]:
len(df_important_speeches45)

66136

In [119]:
df_speeches_master45 = pd.concat([df_dominant_topic45, df_important_speeches45], axis=1, sort=False)
len(df_speeches_master45)

66136

In [120]:
df_speeches_master45=df_speeches_master45.drop(columns=['Text','Unnamed: 0','Document_No','Dominant_Topic','Topic_Perc_Contrib','Keywords','textblob','Speaker','end','important_speaker'])
df_speeches_master45

,category,sentiment,start,Speech,speaker_name,party,name
0,Nonspecific2,0.000000,6087,"The Clerk, I declare my nomination for the office of Speaker.",DOUG KIDD,National,KIDD (NAT)
1,International,0.000000,6190,"The Clerk, I declare my nomination for election as Speaker. \n A personal vote was called for on the question of the election of the Speaker. \n Hon. Doug Kidd 70 Anae English McLauchlan Shipley Awatere Huata Fletcher McLean Shirley Banks Gerard Mapp Simcock Batten Graham Mark Simich Birch Gresham Marshall Smith, L. Bloxham Hasler Maxwell Smith, N. Bolger Henare Morgan Steel Bradford Herlihy Morris te Heuheu Brown Hide Neeson Upton Brownlee Jennings Newman Vernon Carter D Kidd O'Reg...",JONATHAN HUNT,Labour,HUNT (LAB)
2,International,0.265423,7426,"Honourable members, thank you for this appointment---especially those who voted for me. As to those who did not, I want to assure them most sincerely that I will preside without fear or favour, and my impartiality will be assured. With a modicum of cooperation, I believe that this first MMP Parliament will be seen as a watershed. We have now moved into a new system. We have trialled the Standing Orders to operate. Generally they were a very considerable advance and, I believe, a success, bu...",SPEAKERELECT,NaN,NaN
3,International,0.132522,8720,"Mr Speaker-elect, can I be the first to rise to congratulate you on assuming the considerable responsibilities of the Speakership of this House. Now that the vote has been concluded, I am sure I speak on behalf of all members when I say that we will give you our support on most occasions here and will be guided by your advice, although on occasions we might not hear it. Mr Speaker-elect, as you yourself have said, we enter a new era of MMP. We enter it in a House that is much larger, and th...",BOLGER,National,BOLGER (NAT)
4,Nonspecific2,0.010376,10439,"Mr Speaker-elect, I personally offer you congratulations on your election to the position. I express my deep regret that the father of the House was not elected, and I express my considerable concern that the former Deputy Speaker, who served with distinction in the previous Parliament, was not able to have his name put forward. \n I note that the report of the Standing Orders Committee on the new Standing Orders for this Parliament anticipated that in the spirit of MMP there should be ...",HELEN CLARK,Labour,CLARK (LAB)
...,...,...,...,...,...,...,...
66131,International,0.000000,90432640,That is to conclude this debate. Is there any objection? There is none.,SPEAKER,NaN,NaN
66132,Justice,0.143777,90432746,"The honour seems to fall on me to be the last speaker in this 45th Parliament. Let me say at the outset that I can recommend retirement to other MPs because flowers are delivered, and everybody tells you how much you are loved and that they do not want you to go. I have to say that last night I was actually elevated from a back-bench MP to the leader of ACT New Zealand, and even to Prime Minister within the space of one karaoke song. What is more incredible, it all seemed so easily attainab...",PATRICIA SCHNAUER,NaN,NaN
66133,Electoral,0.027037,90442628,"I begin by congratulating Patricia Schnauer on that speech. She has always brought to this House a degree of seriousness and purpose, which has been known to be lacking amongst some of her colleagues sometimes. I particularly regret the fact that her colleague Derek Quigley is not able to be with us today to give his valedictory, as well. Derek Quigley, who has been something of a friend of mine over a large number of years, is somebody who has always taken a high and principled view of pol...",MICHAEL CULLEN,Labour,CULLEN (LAB)
66134,Electoral,0.133774,90443480,"No, it is not, but it is something absolutely unprecedented in parliamentary history. I am doing something today that no MP has ever done before in this Parliament. I am saying farewell to my electorate, while at the same time fully expecting to be re-elected as a member o

In [121]:
df_speeches_master44.to_csv("master44.csv")
df_speeches_master45.to_csv("master45.csv")

In [122]:
df_speeches_master44.head(500)

,category,sentiment,start,Speech,speaker_name,party,name
0,International,1.110774e-01,5166,"I move, That Peter Wilfred Tapsell, the member for Eastern Maori, do take the chair of the House as Speaker. I am honoured and privileged to move this motion, and in doing so I am conscious of the historic significance of this nomination. We who have been elected to serve our country in this Parliament do so at a crucial time in our history. New Zealand has decided to change the method by which MPs are elected, and we in this House will be expected to reflect the new mood in the community i...",JOHN CARTER,National,JOHN CARTER (NAT)
1,Nonspecific,0.000000e+00,6866,"E tu tautoko ana te motini kua whakatakotohia ai i tenei o nga mema o te Tai Tokerau, ara, kia tu ai te mema o te Tai Rawhiti ki runga ki tenei turanga teitei. He tika tonu he tuatahi te honore kua homai nei kia ngai taua ki te iwi Maori, ara, mo Aotearoa katoa. Tuarua, kei te tautoko i nga hau e wha o te motu, tae atu ano ki te Whare nei. \n He mihi kau ake ki te iwi o Te Arawa---Te Arawa waka mai i Maketu ki Tongariro, otira, tena koutou mo te taonga nui kua utaina nei ki runga ki a k...",WETERE,Labour,WETERE (LAB)
2,Nonspecific2,0.000000e+00,10551,Does the member propose to submit to the judgment of the House?,CLERK THE,NaN,NaN
3,Nonspecific2,0.000000e+00,10657,I should like to express my sense of the honour proposed to be conferred on me and to submit myself to the judgment of the House.,PETER TAPSELL,Labour,TAPSELL (LAB)
4,Nonspecific2,0.000000e+00,10813,Are there any further proposals?,CLERK THE,NaN,NaN
5,Nonspecific,0.000000e+00,10879,"I move, That Robin McDowall Gray, the member for Clutha, do take the chair of the House as Speaker.",WINSTON PETERS,NaN,NaN
6,Electoral,1.361685e-01,11059,"He does not miss much, does he? First, I refer to the qualities of dignity and fairness that were cited by the then Senior Government Whip, the Senior Opposition Whip, the Prime Minister, the then Leader of the Opposition, and the leader of the NewLabour Party, when the member for Clutha was picked on 28 November 1990. Those comments are all recorded in Hansard. What has changed over the past 3 years in relation to those statements of adulation made 3 years ago? Do those qualities suddenly ...",WINSTON PETERS,NaN,NaN
7,Electoral,-2.500000e-01,14679,I will keep it up. That is why we have this problem here. The Prime Minister would not listen in the past 3 years; one would think that he might listen now.,WINSTON PETERS,NaN,NaN
8,Electoral,-1.388258e-01,15077,"Is it any wonder that longstanding supporters of Labour, NewLabour, and the Greens are struggling to see the reasons behind the support that has been given by those parties today---their championing of support for the National Party? My former colleagues in the National Party just cannot believe their luck. Just when the electorate had them on the hook, the leadership of \n 21 December 1993 \n \n \n the Alliance and the new leader of the Labour Party can hardly wait to get them off ...",WINSTON PETERS,NaN,NaN
9,Electoral,5.357143e-02,15643,"I second the motion moved by the member for Tauranga that the honourable member for Clutha take the chair as Speaker. It is with a certain amount of objection that I stand here today. It is not an objection to my colleague the honourable member for Tauranga, and it is certainly not an objection to being in these hallowed halls of power, but I am completely in objection to what I consider to be a return to the bad old days of two-party politics. \n On 6 November 1993, as members know, th...",TAU HENARE,NaN,NaN
